# Model Exploration

This notebook is just used to make sure that the model can be run in the pipeline before running the pipeline !

In [51]:
# Copyright 2018-2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# A copy of the License is located at
#
#     https://aws.amazon.com/apache-2-0/
#
# or in the "license" file accompanying this file. This file is distributed
# on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.
from __future__ import absolute_import, division, print_function

import argparse
import json
import logging
import re
import os

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Conv2D, MaxPool2D, BatchNormalization, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2

logging.getLogger().setLevel(logging.INFO)

def keras_model_fn(hyperparameters):
    """keras_model_fn receives hyperparameters from the training job and returns a compiled keras model.
    The model is transformed into a TensorFlow Estimator before training and saved in a
    TensorFlow Serving SavedModel at the end of training.
    """
    _input = Input(shape=eval(hyperparameters["input_shape"]))
    x = _input
    # cnn step
    for _cnn in hyperparameters["cnn"]:
        x = Conv2D(_cnn["filters"], eval(_cnn["kernel"]), activation=_cnn["activation"], padding=_cnn["padding"],
                   kernel_regularizer=l2(hyperparameters["l2_regul"]), bias_regularizer=l2(hyperparameters["l2_regul"]))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D((2, 2))(x)
    # dense step
    x = Flatten()(x)
    for _dense in hyperparameters["dense"]:
        x = Dense(_dense["units"], activation=_dense["activation"],
                 kernel_regularizer=l2(hyperparameters["l2_regul"]), bias_regularizer=l2(hyperparameters["l2_regul"]))(x)
        x = Dropout(hyperparameters["dropout"])(x)
    _output = Dense(hyperparameters["num_classes"], activation="softmax")(x)
    
    # generate the model
    model = Model(inputs=_input, outputs=_output)
    
    print(model.summary())

    # optimizer choice
    if hyperparameters["optimizer"].lower() == "sgd":
        opt = SGD(lr=hyperparameters["learning_rate"], decay=hyperparameters["weight_decay"], momentum=hyperparameters["momentum"])
    elif hyperparameters["optimizer"].lower() == "rmsprop":
        opt = RMSprop(lr=hyperparameters["learning_rate"], decay=hyperparameters["weight_decay"])
    else:
        opt = Adam(lr=hyperparameters["learning_rate"], decay=hyperparameters["weight_decay"])
            
    # compile the model
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=opt,
                  metrics=["accuracy"])
    return model

In [52]:
hyperparameters = json.load(open("../pipeline/hyperparameters.json", "r"))
print(hyperparameters)

{'input_shape': '(28, 28, 1)', 'num_classes': 10, 'epochs': 2, 'batch_size': 64, 'learning_rate': 0.001, 'optimizer': 'adam', 'momentum': 0.9, 'weight_decay': 0.01, 'l2_regul': 0.01, 'dropout': 0.1, 'cnn': [{'filters': 32, 'kernel': '(3, 3)', 'activation': 'relu', 'padding': 'same'}, {'filters': 32, 'kernel': '(3, 3)', 'activation': 'relu', 'padding': 'same'}], 'dense': [{'units': 128, 'activation': 'relu'}]}


In [53]:
model = keras_model_fn(hyperparameters)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_16 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
batch_normalization_17 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 32)          0   

In [54]:
import numpy as np
model(np.ones(shape=(1, 28, 28, 1), dtype=np.float32))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.10145849, 0.10281526, 0.09943226, 0.11487889, 0.09337983,
        0.10688938, 0.09324344, 0.08578034, 0.10605171, 0.09607042]],
      dtype=float32)>